# Kurs i kontrollfunksjoner

Dette er et kurs i kontrollfunksjoner. Pakken validate skal brukes til å sette opp regler, kjøre kontroller og analysere resultater.

### R-pakker som skal brukes 

In [ ]:
library(validate)
library(klassR)
library(Kostra)
library(plotly)

# Oppgaver valideing 
Oppgave 1. Last inn datasettet kirkedata.R og se på datasettet

Variabler på datasettet:
* region - kommune nummer
* Tid - år for statistkken
* dopte - antall døpte i kommunen
* fodde - antall fødte i kommunen
* konfirmanter - antall konfirmanter i kommunen
* personer15 - antall personer som er 15 år
* Tid_1 - forrige år
* dopte - antall døpte i kommunen forrige år
* fodde - antall fødte i kommunen forrige år
* konfirmanter - antall konfirmanter i kommunen forrige år
* personer15 - antall personer som er 15 år forrige år
* name - navn på kommunen
* kostragr - kommuneinndeling brukt i kostra

In [ ]:
load("kirkedata.RData")
head(kirkedata)

Oppgave 2. Sett opp noen kontroller for antall døpte

In [ ]:
regler<-validator( dopte>0,
                   dopte<=fodde)

Oppgave 3. Kjør kontrollene på datasettet kirkedata

In [ ]:
resultat<-confront(kirkedata,regler,key="region")

Oppgave 4. Analyser kontrollene

In [ ]:
summary(resultat)
aggregate(resultat)
plot(resultat)

Ekstra oppgave. Last kommunelista ned fra Klass og kontroller at det er gyldige kommuner

In [ ]:
sn <- GetKlass(klass = 131, date = "2019-01-01") 
komliste<-as.vector(sn[,c("code")])
regler<-validator( region %in% komliste)
resultat<-confront(kirkedata,regler,key="region")
summary(resultat)

# Kontrollfunksjoner Kostra-pakken

Oppgave 5  Last inn datasettet kirkedata_0.R

In [ ]:
load("kirkedata_0.RData")

Oppgave 6 Sjekk om det er tusedenfeil med funksjoen ThError i rapportering av antall døpte.
Lag figurer eller print ut resulatet 

In [ ]:
tusres<-ThError(data = kirkedata_0, id = "region", x1 = "dopte", x2 = "dopte_1",
                ll = -3, ul = 3)
#Plotter punktene, med forklaringer på aksene og tittel
plot_ly(data = tusres,x= ~x1, y=~diffLog10, type ="scatter", split = ~outlier,mode ="markers",
                text = paste("<br><br>Id:  ",tusres$id, "<br><br>Verdi i år:  ", tusres$x1,"<br>Verdi i fjor:", tusres$x2 ),            
                hovertemplate = paste( "<b>%{text}<br>", "Logaritmen til differansen: %{y:}<br>", "<extra></extra>" )              )   %>% 
  layout(title = "Tusenfeil", xaxis = list(title = "Døpte"), yaxis = list(title = "logaritmen til differansen"), legend=list(title=list(text='Outlier:'))) %>%
  add_trace(x = ~x1, y=~upperLimit,inherit = FALSE, name = 'Øvre grense' , mode="lines", type="scatter")  %>% 
  add_trace(x = ~x1, y=~lowerLimit,inherit = FALSE, name = 'Nedre grense',mode="lines", type="scatter") 


Oppgave 7 Sjekk om du finner noen merkelige observasjoner med HB-metoden (Hb), varier parametre C og U.

In [ ]:
hbres <- Hb(data = kirkedata_0, id = "region", x1 = "dopte", x2 = "dopte_1",pC=8,pU=0.6,pA=0.05)

#Dataene må være sortert for at grafen skal bli korrekt
hbres<-hbres[order(hbres$maxX),]

#Plotter punktene, med forklaringer på aksene og tittel


plot_ly(data = hbres,x= ~maxX, y=~ratio, type ="scatter",split = ~outlier, mode ="markers",
        text = paste("<br><br>Id:  ",hbres$id, "<br><br>Verdi i år:  ", hbres$x1,"<br>Verdi i fjor:", hbres$x2 ),            
        hovertemplate = paste( "<b>%{text}<br>", "Forholdstallet: %{y:}<br>", "<extra></extra>" )   
       )  %>% 
  layout(title = "HB metoden Døpte i kommunen",
         xaxis = list(title = "Maks antall døpte"),
         yaxis = list(title = "Forholdstallet"),
         legend=list(title=list(text='Outlier:'))
  ) %>% 
  add_trace(x = ~maxX, y=~upperLimit, name = 'Øvre grense' , mode="lines", inherit = FALSE, type="scatter")  %>% 
  add_trace(x = ~maxX, y=~lowerLimit, name = 'Nedre grense',mode="lines", inherit = FALSE, type="scatter") 



 Oppgave 8 Test ut kvartilmetoden Quartilemed  forskjellige grenseverdier på forholdet mellom døpte og fødte i en kommune

In [ ]:
qres <- Quartile(data = kirkedata_0, id = "region", x1 = "dopte", y1 = "fodde",pKL=3,pKU=3)
plot_ly(data = qres,x= ~ratio, type ="histogram", name="observasjon")  %>% 
  layout(title = "Kvartilmetode",
         xaxis = list(title = "Andel døpte per fødte"),
         yaxis = list(title = "Antall kommuner"))%>% 
  add_segments(x = ~upperLimit, xend=~upperLimit,y=0, yend=50, name = 'Øvre grense' ) %>% 
  add_segments(x = ~lowerLimit, xend=~lowerLimit,y=0, yend=50,name = 'Øvre grense' ) 


Oppgave 9 Test ut robust regresjon (OutlierRegressionMicro) med forskjellige modeller og grenser på forholde mellom døpte og fødte

In [ ]:
regres <- OutlierRegressionMicro(data= kirkedata_0, idName ="region" , strataName = NULL,
                                 xName ="fodde" , yName ="dopte" ,
                                 method = "ordinary", limitModel = 5, limitIterate = 6)

#Plotter punktene, med forklaringer på aksene og tittel
plot_ly(data = regres,x= ~x, y=~y, type ="scatter", mode ="markers", split = ~outlier,
                text = paste("<br><br>Id:  ",regres$id, "<br><br>fødte :  ", regres$x,"<br>Døpte:", regres$y ),            
                hovertemplate = paste( "<b>%{text}<br>" )  
)  %>% 
  layout(title = "Robust regresjon døpte mot fødte i kommunen",
         xaxis = list(title = "Antall fødte"),
         yaxis = list(title = "Antall døpte"),
         legend=list(title=list(text='Outlier:'))
  ) %>% 
  add_lines(x = ~x, y=~yHat, name="Linje", inherit = FALSE, type="scatter")  


Oppgave 10 Test ut funksjonen Rank2NumVar for å finne innflytelse på totalen for antall døpte

In [ ]:
rankres<-Rank2NumVar(data=kirkedata_0, idVar= "region", xVar="dopte", yVar= "dopte_1", strataVar =NULL, 
                     antall = 10,grense = NULL, identiske = FALSE)
rankres

Oppgave 11 Test ut funksjonen Diff2NumVar for å finne innflytelse på endringstallet

In [ ]:
diffres<- Diff2NumVar(data=kirkedata_0, idVar= "region", xVar="dopte_1", yVar="dopte", strataVar = NULL, antall = 5,
                      grense = NULL, zVar = NULL, kommentarVar = NULL)

diffres[,c("id","x","y","Diff","DiffProsAvTotx") ]

Oppgave 12 Test ut analyse av aggregat med funksjonen AggrSml2NumVar

In [ ]:
aggres<- AggrSml2NumVar(data=kirkedata_0, xVar="dopte_1", yVar="dopte", strataVar = "kostragr", identiske = FALSE)
aggres

plot_ly(data=aggres, x = ~strata,  y = ~Sumx, name = "Døpte forrige år", type = "bar") %>% 
  add_trace(y = ~Sumy, name = 'Døpte i år')


plot_ly(data=aggres, x =~strata ,  y = ~Diff, type = "bar")
